In [ ]:
# ! pip install ogb
# ! python -c "import ogb; print(ogb.__version__)"
# # Otherwise, please update the version by running
# ! pip install -U ogb

# ! pip install torch_geometric
# ! pip install --upgrade ogb

# ! pip install -U torch-geometric
# ! pip install torch-scatter torch-sparse torch-cluster



In [ ]:
# import os
# import shutil

# processed_dir = 'dataset/ogbl_collab/processed'
# if os.path.exists(processed_dir):
#     shutil.rmtree(processed_dir)

from ogb.linkproppred import PygLinkPropPredDataset
from torch_geometric.loader import DataLoader

# from ogb.graphproppred import PygGraphPropPredDataset
# from torch_geometric.data import DataLoader

# Download and process data, stored in './dataset/ogbl_collab/'
dataset = PygLinkPropPredDataset(name="ogbl-collab", root='dataset/')


In [ ]:
# Get split edges from the dataset
split_edge = dataset.get_edge_split()

# Extract edge lists for train, valid, and test splits
train_edges = split_edge["train"]['edge']  # List of training edges
valid_edges = split_edge["valid"]['edge']  # List of validation edges
test_edges = split_edge["test"]['edge']    # List of test edges

from torch_geometric.data import Data

# Convert to Data objects if necessary
train_data = Data(edge_index=train_edges.T)  # Transpose to match PyG format
valid_data = Data(edge_index=valid_edges.T)
test_data = Data(edge_index=test_edges.T)

# Create DataLoaders
train_loader = DataLoader([train_data], batch_size=32, shuffle=True)
valid_loader = DataLoader([valid_data], batch_size=32, shuffle=False)
test_loader = DataLoader([test_data], batch_size=32, shuffle=False)

# Load graph data
data = dataset[0]  # PyG graph object


In [ ]:
from ogb.graphproppred import Evaluator

evaluator = Evaluator(name = "ogbg-molhiv")
# You can learn the input and output format specification of the evaluator as follows.
# print(evaluator.expected_input_format) 
# print(evaluator.expected_output_format) 
input_dict = {"y_true": y_true, "y_pred": y_pred}
result_dict = evaluator.eval(input_dict) # E.g., {"rocauc": 0.7321}